In [3]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

import keras

from keras import layers, callbacks, optimizers, mixed_precision
from keras.applications import EfficientNetB5

mixed_precision.set_global_policy('mixed_float16')

In [4]:
# Append the absolute file path to the CSV files to fetch the training and testing data.
def append_path(df, path):
    # Append .jpg extenstion
    def append_ext(fn):
        return fn + ".jpg"

    df['image'] = df['image'].apply(append_ext)
    print("Dataframe Head:\n", df['image'].head())

    # Append absolute file path to where the images are located. 
    abs_file_names = []
    for file_name in df['image']:
        tmp = path + '/' + file_name
        abs_file_names.append(tmp)

    df['image'] = abs_file_names
    print("Updated DF with Extenstion and Path:\n", df['image'][0])

    return df

In [5]:
def cosine_decay_with_warmup(global_step,
                                learning_rate_base,
                                total_steps,
                                warmup_learning_rate=0.0,
                                warmup_steps=0,
                                hold_base_rate_steps=0):

    global_step = tf.cast(global_step, tf.float32)
    total_steps = tf.cast(total_steps, tf.float32)
    warmup_steps = tf.cast(warmup_steps, tf.float32)
    hold_base_rate_steps = tf.cast(hold_base_rate_steps, tf.float32)

    # Linear warmup
    slope = (learning_rate_base - warmup_learning_rate) / tf.maximum(warmup_steps, 1.0)
    warmup_rate = slope * global_step + warmup_learning_rate

    # Cosine decay
    cosine_steps = total_steps - warmup_steps - hold_base_rate_steps
    cosine_steps = tf.maximum(cosine_steps, 1.0)

    cosine_global_step = tf.minimum(global_step - warmup_steps - hold_base_rate_steps,
                                    cosine_steps)

    cosine_decay = 0.5 * learning_rate_base * (
        1 + tf.cos(np.pi * cosine_global_step / cosine_steps)
    )

    # Conditions
    lr = tf.where(global_step < warmup_steps, warmup_rate, cosine_decay)
    lr = tf.where(global_step > total_steps, 0.0, lr)

    return lr

In [6]:
class WarmUpCosineDecaySchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 name=None):
        super().__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.name = name

    def __call__(self, step):
        return cosine_decay_with_warmup(
            global_step=step,
            learning_rate_base=self.learning_rate_base,
            total_steps=self.total_steps,
            warmup_learning_rate=self.warmup_learning_rate,
            warmup_steps=self.warmup_steps,
            hold_base_rate_steps=self.hold_base_rate_steps
        )

    def get_config(self):
        return {
            "learning_rate_base": self.learning_rate_base,
            "total_steps": self.total_steps,
            "warmup_learning_rate": self.warmup_learning_rate,
            "warmup_steps": self.warmup_steps,
            "hold_base_rate_steps": self.hold_base_rate_steps,
            "name": self.name,
        }

In [7]:
def save_plot(history, save_dir):

    # Accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(save_dir + '_accuracy.jpg')
    plt.close()

    # Loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(save_dir + '_loss.jpg')
    plt.close()

In [8]:
def model_parameter():
    param = {
            "backbone": EfficientNetB5,
            "target": 9,
            "resize": 448,
            "metadata": False,
            "initial_lr": 3e-5,
            "epochs": 15,
            'input_image_size': 512,
            'train_batch_size': 4,
            'validation_batch_size': 4,
            "savedModelByName": "Model.keras",
            "saveFinalModelBy": "Model",
            'log_by': "Model.csv",
            'save_plot_name': 'Model',
            'prediction_csv_name': 'Model_prediction',
            'print_hyper_parameter': True,
            'print_trainable_layers': False,
            'print_model_summary': False,
            'visualise_augmented_data': False
            }
    return param

In [9]:
# Initialise the EfficientNet Model for transfer learning
def EffNet(input_size, num_classess, pretrained_model,
           print_trainable_layers=False, print_model_summary=False):
    # Get the EfficientNet Model
    base_model = pretrained_model(
        # weights='imagenet',
        weights="/kaggle/input/efficientb5/efficientnetb5_notop.h5",
        input_shape=input_size,
        include_top=False
    )

    # Keep the BatchNorm layer freeze, and unfreeze all other layers
    def unfreeze_model(model, print_trainable, print_summary):
        # unfreeze the layers while leaving BatchNorm layers frozen
        for layer in model.layers[:]:
            if isinstance(layer, layers.BatchNormalization):
                layer.trainable = False
                
        # Print trainable layer summary
        if print_trainable:
            for layer in model.layers:
                print(layer, layer.trainable)

        # Print Model summary
        if print_summary:
            model.summary()

    # Unfreeze the model
    unfreeze_model(base_model, print_trainable_layers, print_model_summary)

    # Add dense and output layer
    model = keras.Sequential([
    base_model,
    layers.Flatten(name="top_flatten"),
    layers.Dense(500, activation="relu", name="dense_500"),
    layers.Dense(256, activation="relu", name="dense_256"),
    layers.Dense(num_classess, activation="softmax", name="output_layer"),
    ])


    # Initialise the optimizer and compile the model
    lr_schedule = WarmUpCosineDecaySchedule(
    learning_rate_base=hyper_param['learning_rate_base'],
    total_steps=total_steps,
    warmup_learning_rate=hyper_param['warmup_learning_rate'],
    warmup_steps=warmup_steps,
    hold_base_rate_steps=0,
    )

    optimizer = optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


    # print the FC layer summary
    if print_model_summary:
        model.summary()

    return model

In [10]:
# Fit the model on training and validation dataset and star the training process.
def train_model(model, train_dataset, epoch,
                validation_dataset, callback):

    return model.fit(
        train_dataset,
        epochs=epoch,
        validation_data=validation_dataset,
        verbose=1,
        callbacks=callback
    )

In [11]:
# Augment the dataset
IMAGENET_MEAN = tf.constant([0.485, 0.456, 0.406], dtype=tf.float32)
IMAGENET_STD  = tf.constant([0.229, 0.224, 0.225], dtype=tf.float32)
def augment(image, image_size, training=True):


    image = tf.image.convert_image_dtype(image, tf.float32)

    if training:

        rand_scale = tf.random.uniform([], 0.8, 1.0)
        orig_h = tf.shape(image)[0]
        orig_w = tf.shape(image)[1]
        crop_h = tf.cast(rand_scale * tf.cast(orig_h, tf.float32), tf.int32)
        crop_w = tf.cast(rand_scale * tf.cast(orig_w, tf.float32), tf.int32)
    
        image = tf.image.random_crop(image, size=[crop_h, crop_w, 3])
        image = tf.image.resize(image, [image_size, image_size])
    
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
    
        k = tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32)
        image = tf.image.rot90(image, k=k)
    
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
        image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
        image = tf.image.random_hue(image, max_delta=0.02)
    
        image = tf.clip_by_value(image, 0.0, 1.0)
    
        noise_std = tf.random.uniform([], 0.0, 0.05)  
        noise = tf.random.normal(tf.shape(image), mean=0.0, stddev=noise_std)
        image = image + noise
        image = tf.clip_by_value(image, 0.0, 1.0)

        def apply_cutout(img):
            h = tf.shape(img)[0]
            w = tf.shape(img)[1]
        
      
            cutout_frac = tf.random.uniform([], 0.2, 0.4)
            ch = tf.cast(cutout_frac * tf.cast(h, tf.float32), tf.int32)
            cw = tf.cast(cutout_frac * tf.cast(w, tf.float32), tf.int32)
        
    
            cy = tf.random.uniform([], 0, h - ch + 1, dtype=tf.int32)
            cx = tf.random.uniform([], 0, w - cw + 1, dtype=tf.int32)
        
    
            mask = tf.ones((h, w), dtype=tf.float32)
            mask = tf.tensor_scatter_nd_update(
                mask,
                indices=tf.reshape(
                    tf.stack(tf.meshgrid(tf.range(cy, cy + ch),
                                         tf.range(cx, cx + cw),
                                         indexing="ij"), axis=-1),
                    [-1, 2]
                ),
                updates=tf.zeros([ch * cw], dtype=tf.float32)
            )
        
    
            mask = tf.expand_dims(mask, -1)
            mask = tf.concat([mask, mask, mask], axis=-1)
        
            return img * mask
    
        do_cutout = tf.less(tf.random.uniform([]), 0.5)
        image = tf.cond(do_cutout, lambda: apply_cutout(image), lambda: image)

    else:
        image = tf.image.resize(image, [image_size, image_size])
    
    image = (image - IMAGENET_MEAN) / IMAGENET_STD
    
    return image


I0000 00:00:1765208886.378971      94 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [12]:
AUTOTUNE = tf.data.AUTOTUNE

def build_tf_dataset(
    df,
    image_size,
    batch_size,
    num_classes,
    label_col="diagnosis_idx",
    shuffle=False,
    repeat=False,
    training=True,
):
    paths = df["image"].values
    labels = df[label_col].values.astype("int32")

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    if shuffle:
        ds = ds.shuffle(len(df), reshuffle_each_iteration=True)

    def _load_image(path, label):
        image_bytes = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image_bytes, channels=3)
        image = augment(image, image_size, training=training)
        label_onehot = tf.one_hot(label, num_classes)
        return image, label_onehot

    ds = ds.map(_load_image, num_parallel_calls=AUTOTUNE)

    if repeat:
        ds = ds.repeat()

    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)
    return ds


In [13]:
def build_tf_test_dataset(df, image_size, batch_size):
    paths = df["image"].values
    ds = tf.data.Dataset.from_tensor_slices(paths)

    def _load_image(path):
        image_bytes = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image_bytes, channels=3)
        image = augment(image, image_size, training=False)  
        return image

    ds = ds.map(_load_image, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)
    return ds

In [14]:
if __name__ == "__main__":
    # Data path
    TRAIN_CSV = "/kaggle/input/processedcsvs/Processed CSV's/train_2020_and_2019_with_9_Labels.csv"
    TEST_CSV  = "/kaggle/input/processedcsvs/Processed CSV's/test_2020_no_PateintDetail.csv"
    TRAIN_IMG_DIR_1 = "/kaggle/input/siim-isic-melanoma-classification/jpeg/train"
    TRAIN_IMG_DIR_2 = "/kaggle/input/isic-2019/ISIC_2019_Training_Input/ISIC_2019_Training_Input"
    TEST_IMG_DIR = "/kaggle/input/siim-isic-melanoma-classification/jpeg/test"

    #  load model parameter
    selected_model = model_parameter()

    # Define log, plot, model, prediction files
    log_path = "./runs/"
    os.makedirs(log_path, exist_ok=True)

    plot_path = "./plot/"
    os.makedirs(plot_path, exist_ok=True)

    save_model_path = "./saveModel/"
    os.makedirs(save_model_path, exist_ok=True)

    prediction_path = "./prediction/"
    os.makedirs(prediction_path, exist_ok=True)

    # join CSV & image path
    label = pd.read_csv(TRAIN_CSV)
    class_names = sorted(label["diagnosis"].unique())
    label2idx = {c: i for i, c in enumerate(class_names)}
    idx2label = {i: c for c, i in label2idx.items()}

    label["diagnosis_idx"] = label["diagnosis"].map(label2idx)
    num_classes = len(class_names)
    test_csv = pd.read_csv(TEST_CSV)

    def attach_train_paths(df):
        df = df.copy()
        df["image"] = df["image"].astype(str) + ".jpg"

        full_paths = []
        miss_cnt = 0

        for img in df["image"]:
            p1 = os.path.join(TRAIN_IMG_DIR_1, img)  # 2020
            p2 = os.path.join(TRAIN_IMG_DIR_2, img)  # 2019

            if os.path.exists(p1):
                full_paths.append(p1)
            elif os.path.exists(p2):
                full_paths.append(p2)
            else:
                full_paths.append(p1)
                miss_cnt += 1

        if miss_cnt > 0:
            print(f"[WARNING] {miss_cnt} images not found in either train dir, "
                  f"using 2020 path by default. Check names or dirs.")
            
        df["image"] = full_paths
        return df

    label = attach_train_paths(label)

    def attach_test_paths(df):
        df = df.copy()
        df["image"] = df["image"].astype(str) + ".jpg"
        df["image"] = df["image"].apply(
            lambda x: os.path.join(TEST_IMG_DIR, x)
        )
        return df
    test_csv = attach_test_paths(test_csv)

    # Hyper Parameter
    hyper_param = {
        'seed': 42,
        'image_size': selected_model['resize'],
        'backbone_model': selected_model['backbone'],
        'early_stop': 10,
        'num_class': selected_model['target'],
        'train_batch_size': selected_model['train_batch_size'],
        'test_batch_size': 1,
        'validation_batch_size': selected_model['validation_batch_size'],
        'epoch': selected_model['epochs'],
        'warmup_epoch': 1,
        'learning_rate_base': selected_model['initial_lr'],
        'warmup_learning_rate': selected_model['initial_lr'],
        'training_sample_count': label.shape[0],
        'save_model': selected_model['savedModelByName'],
        'save_final_model': selected_model['saveFinalModelBy']
    }

    image_resize = (hyper_param['image_size'], hyper_param['image_size'])
    image_shape = image_resize + (3,)

    # Total training steps in Warmup
    total_steps = int(hyper_param['epoch'] *
                      hyper_param['training_sample_count'] /
                      hyper_param['train_batch_size'])
    # Compute the number of warmup batches.
    warmup_steps = int(hyper_param['warmup_epoch'] *
                       hyper_param['training_sample_count'] /
                       hyper_param['train_batch_size'])

    # Print Hyper parameter
    if selected_model['print_hyper_parameter']:
        print("\n####################### Hyper Parameter #################################\n")
        pprint.pprint(hyper_param)
        print('\nImage Shape: {}'.format(image_shape))
        print('Total training steps in Warmup: {}'.format(total_steps))
        print('Number of Warmup Batch: {}\n'.format(warmup_steps))
        print("\nTrain Label shape: ", label.shape)
        print("Test Label shape: ", test_csv.shape)


    #Initialise Pre-train Model
    # model = EffNet(
    #     input_size=image_shape,
    #     num_classess=hyper_param['num_class'],
    #     pretrained_model=hyper_param['backbone_model'],
    #     print_trainable_layers=selected_model['print_trainable_layers'],
    #     print_model_summary=selected_model['print_model_summary']
    # )
    model = tf.keras.models.load_model("/kaggle/input/saved/keras/default/1/Model.keras",compile=False)
    optimizer = tf.keras.optimizers.Adam(learning_rate=hyper_param['learning_rate_base'])
    model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])

    # Preprocess and Augment Image for train, test and validation set. 
    image_size = hyper_param['image_size']
    train_bs = hyper_param['train_batch_size']
    val_bs = hyper_param['validation_batch_size']
    train_df, val_df = train_test_split(label,test_size=0.2,random_state=hyper_param['seed'],stratify=label['diagnosis'])


    # Prepare train, validation Generator
    train_ds = build_tf_dataset(
        df=train_df,
        image_size=image_size,
        batch_size=train_bs,
        num_classes=hyper_param['num_class'],
        label_col="diagnosis_idx",
        shuffle=True,
        repeat=False,
        training=True,      
    )
    
    val_ds = build_tf_dataset(
        df=val_df,
        image_size=image_size,
        batch_size=val_bs,
        num_classes=hyper_param['num_class'],
        label_col="diagnosis_idx",
        shuffle=False,
        repeat=False,
        training=False,    
    )
    # Define Early Stopping on validation loss
    es = callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=hyper_param['early_stop'],
        verbose=1,
        restore_best_weights=True
    )

    # Save model after each epoch
    ck = callbacks.ModelCheckpoint(
        filepath=os.path.join(save_model_path, hyper_param['save_model']),
        monitor='val_loss',
        verbose=1,
        save_best_only=False,
        save_weights_only=False,
    )

    # Save logs to CSV
    logs = callbacks.CSVLogger(
        os.path.join(log_path, selected_model['log_by']),
        separator=",",
        append=False
    )

    # Callback list
    call_backs = [ck, logs, es]

    already_trained= 12

    # Start the training process
    print('\n\n---------------- Staring the Training Process... --------------- ')
    history = model.fit(
        train_ds,
        epochs=hyper_param['epoch'],
        initial_epoch=already_trained,
        validation_data=val_ds,
        callbacks=call_backs,
        verbose=1
    )
    print("\n ----------------- Model is trained --------------------------")

    # Training and validation: accuracy & loss
    print("\n------ Saving Training and Validation Plot --------")
    save_plot(history=history,
              save_dir=os.path.join(plot_path, selected_model['save_plot_name']))

    # Predict on Testing Set
    test_ds = build_tf_test_dataset(
        df=test_csv,
        image_size=image_size,
        batch_size=hyper_param['test_batch_size'],         
    )

    print("\n------ Predicting on Testset --------")
    pred = model.predict(test_ds, verbose=1)
    predicted_class_indices = np.argmax(pred, axis=1)

    # Map the predicted labels with their unique ids
    predictions = [idx2label[i] for i in predicted_class_indices]

    results = pd.DataFrame({
        "Filename": test_csv["image"],   
        "Predictions": predictions
    })

    results.to_csv(
        os.path.join(prediction_path, selected_model['prediction_csv_name'] + ".csv"),
        index=False
    )

    # Save Trained Model
    print("\n ------------ Saving the Trained model ------------------------------------")
    final_model_dir = os.path.join(save_model_path, hyper_param['save_final_model'])
    os.makedirs(final_model_dir, exist_ok=True)

    model.save(final_model_dir + ".keras", include_optimizer=True)

    print("\n---------------------- Completed Model Training & Prediction ---------------------------")


####################### Hyper Parameter #################################

{'backbone_model': <function EfficientNetB5 at 0x7a79bd45cae0>,
 'early_stop': 10,
 'epoch': 15,
 'image_size': 448,
 'learning_rate_base': 3e-05,
 'num_class': 9,
 'save_final_model': 'Model',
 'save_model': 'Model.keras',
 'seed': 42,
 'test_batch_size': 1,
 'train_batch_size': 4,
 'training_sample_count': 58031,
 'validation_batch_size': 4,
 'warmup_epoch': 1,
 'warmup_learning_rate': 3e-05}

Image Shape: (448, 448, 3)
Total training steps in Warmup: 217616
Number of Warmup Batch: 14507


Train Label shape:  (58031, 3)
Test Label shape:  (10875, 1)


---------------- Staring the Training Process... --------------- 
Epoch 13/15


I0000 00:00:1765209288.576051     144 service.cc:148] XLA service 0x7a7914002130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765209288.577087     144 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1765209298.736842     144 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1765209358.481746     144 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11606/11606 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - accuracy: 0.8426 - loss: 0.4664
Epoch 13: saving model to ./saveModel/Model.keras
11606/11606 ━━━━━━━━━━━━━━━━━━━━ 4658s 390ms/step - accuracy: 0.8426 - loss: 0.4664 - val_accuracy: 0.8464 - val_loss: 0.4448
Epoch 14/15
11606/11606 ━━━━━━━━━━━━━━━━━━━━ 0s 337ms/step - accuracy: 0.8484 - loss: 0.4400
Epoch 14: saving model to ./saveModel/Model.keras
11606/11606 ━━━━━━━━━━━━━━━━━━━━ 4530s 390ms/step - accuracy: 0.8484 - loss: 0.4400 - val_accuracy: 0.8576 - val_loss: 0.4209
Epoch 15/15
11606/11606 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.8545 - loss: 0.4233
Epoch 15: saving model to ./saveModel/Model.keras
11606/11606 ━━━━━━━━━━━━━━━━━━━━ 4565s 393ms/step - accuracy: 0.8545 - loss: 0.4233 - val_accuracy: 0.8597 - val_loss: 0.4217
Restoring model weights from the end of the best epoch: 14.

 ----------------- Model is trained --------------------------

------ Saving Training and Validation Plot --------

------ Predicting on Testse

E0000 00:00:1765222997.494456     146 buffer_comparator.cc:157] Difference at 6: 1808, expected 2009
E0000 00:00:1765222997.495530     146 buffer_comparator.cc:157] Difference at 9: 1826, expected 2033
E0000 00:00:1765222997.495544     146 buffer_comparator.cc:157] Difference at 13: 1822, expected 2027
E0000 00:00:1765222997.495548     146 buffer_comparator.cc:157] Difference at 14: 1817, expected 2023
E0000 00:00:1765222997.495551     146 buffer_comparator.cc:157] Difference at 20: 1817, expected 2024
E0000 00:00:1765222997.495555     146 buffer_comparator.cc:157] Difference at 23: 1825, expected 2028
E0000 00:00:1765222997.495558     146 buffer_comparator.cc:157] Difference at 24: 1820, expected 2024
E0000 00:00:1765222997.495562     146 buffer_comparator.cc:157] Difference at 29: 1826, expected 2030
E0000 00:00:1765222997.495565     146 buffer_comparator.cc:157] Difference at 30: 1813, expected 2015
E0000 00:00:1765222997.495569     146 buffer_comparator.cc:157] Difference at 34: 18

10875/10875 ━━━━━━━━━━━━━━━━━━━━ 825s 74ms/step

 ------------ Saving the Trained model ------------------------------------

---------------------- Completed Model Training & Prediction ---------------------------
